In [3]:
import pandas as pd
import altair as alt
import re
from datetime import datetime

# Load the data
df = pd.read_csv('karcher_reviews.csv')

# Function to convert relative date to actual date
def convert_relative_date(date_str, reference_date):
    if 'day ago' in date_str or 'days ago' in date_str:
        days = 1 if date_str == 'a day ago' else int(re.search(r'\d+', date_str).group())
        return reference_date - pd.Timedelta(days=days)
    elif 'month ago' in date_str or 'months ago' in date_str:
        months = 1 if date_str == 'a month ago' else int(re.search(r'\d+', date_str).group())
        return reference_date - pd.DateOffset(months=months)
    elif 'year ago' in date_str or 'years ago' in date_str:
        years = 1 if date_str == 'a year ago' else int(re.search(r'\d+', date_str).group())
        return reference_date - pd.DateOffset(years=years)
    else:
        return pd.to_datetime(date_str)

# Set the reference date (today's date in the context)
reference_date = datetime(2024, 6, 29)

# Create new date column
df['actual_date'] = df['date'].apply(lambda x: convert_relative_date(x, reference_date))

# Convert to datetime and extract year-month
df['year_month'] = df['actual_date'].dt.to_period('M').astype(str) + '-01'
df['year_month'] = pd.to_datetime(df['year_month'])

# Count reviews per month
reviews_per_month = df.groupby('year_month').size().reset_index(name='count')

# Calculate the average number of reviews
avg_reviews = reviews_per_month['count'].mean()

# Create the Altair chart
base = alt.Chart(reviews_per_month).encode(
    x=alt.X('year_month:T', axis=alt.Axis(title='', format='%Y-%m', labelAngle=-45, labelOverlap=True)),
    y=alt.Y('count:Q', axis=alt.Axis(title='Number of Reviews', titleAngle=270, labelOverlap=True))
)

line = base.mark_line(color='#5276A7', interpolate='monotone').encode(
    tooltip=[
        alt.Tooltip('year_month:T', title='Date', format='%Y-%m'),
        alt.Tooltip('count:Q', title='Number of Reviews')
    ]
)

points = base.mark_circle(color='#5276A7', size=60).encode(
    opacity=alt.condition(alt.datum.count == reviews_per_month['count'].max(), alt.value(1), alt.value(0))
)

text = base.mark_text(align='left', dx=5, dy=-10, color='#5276A7').encode(
    text='count:Q',
    opacity=alt.condition(alt.datum.count == reviews_per_month['count'].max(), alt.value(1), alt.value(0))
)

rule = alt.Chart(pd.DataFrame({'average': [avg_reviews]})).mark_rule(color='#E15759', strokeDash=[3, 3]).encode(
    y='average:Q'
)

chart = (line + points + text + rule).properties(
    width=800,
    height=400,
    title={
        "text": ["Kärcher Review Volume Peaks in Recent Months", "Number of customer reviews per month"],
        "subtitle": ["Average: {:.0f} reviews per month".format(avg_reviews)],
        "align": "left",
        "anchor": "start",
        "color": "black",
        "subtitleColor": "gray"
    }
).configure_axis(
    grid=False
).configure_view(
    strokeWidth=0
)

# Display the chart
chart.show()

# Print statistics
print(f"Total number of reviews: {len(df)}")
print(f"Date range: from {df['actual_date'].min().strftime('%Y-%m-%d')} to {df['actual_date'].max().strftime('%Y-%m-%d')}")
print(f"Month with most reviews: {reviews_per_month.loc[reviews_per_month['count'].idxmax(), 'year_month'].strftime('%Y-%m')} ({reviews_per_month['count'].max()} reviews)")
print(f"Month with least reviews: {reviews_per_month.loc[reviews_per_month['count'].idxmin(), 'year_month'].strftime('%Y-%m')} ({reviews_per_month['count'].min()} reviews)")
print(f"Average reviews per month: {avg_reviews:.2f}")

alt.LayerChart(...)

Total number of reviews: 3243
Date range: from 2019-06-29 to 2024-06-28
Month with most reviews: 2020-06 (936 reviews)
Month with least reviews: 2023-07 (20 reviews)
Average reviews per month: 190.76
